# Introduction
Series of sample Salesforce CLI commands demonstrating common tasks with Salesforce database service.

# Demo: Local Environment Setup

## Create Accounts and Install Tools
1. Sign up for free [Developer Edition](https://developer.salesforce.com/signup) account
1. Install the [Salesforce CLI](https://developer.salesforce.com/tools/sfdxcli)
1. (Optionally) [Install VS Code](https://code.visualstudio.com/download), and the [Salesforce extension pack](https://marketplace.visualstudio.com/items?itemName=salesforce.salesforcedx-vscode)

## Connect Salesforce CLI to your DevHub
The Salesforce instance used for the DevHub will determine some of the features and capabilities that the Salesforce CLI has access to and this step is required in order to run many of the commands.

Use your developer edition credentials when prompted by the command below.
```
sfdx auth:web:login -d -a myDevHub
```
Enable DevHub in your Developer Edition org

In [ ]:
! sfdx force:org:open -u myDevHub

## Create a Scratch Org
A scratch org is a temporary Salesforce instance available for up to 30 days, intended to be used with source-driven development.

A local project with configuration files and source code is required in order to create a scratch org. In this example we will clone an existing project from GitHub.
```
git clone https://github.com/aaronwinters/sql-guide-to-salesforce.git
```
Next create the scratch org. These are the options included in the command below
* `-s` option indicates that this scratch org should be the default org for this project when running Salesforce CLI commands.
* `-f` is the path to the configuration file that specifies which features we want available in the scratch org.
* `-a` creates an alias for the org that we can use to easily reference this org from the CLI.

In [ ]:
! sfdx force:org:create -s -f orgs/dev.json -a AnimalAdoptionScratch

# Demo: Create & Update Schema
## Push Project Metadata to the Scratch org
Typically the data model is created via a web application within the Salesforce scratch org.

In this example, the project we cloned from GitHub already has the schema metadata, so we will push that to the sratch org.

In [ ]:
! sfdx force:source:push

## Grant Scratch Org User Access to the Objects and Fields
By default, users don't have access to the objects and fields. One commonly used tool to manage access is called Permission Sets. 

Assign the permission set to your scratch org user.

In [ ]:
! sfdx force:user:permset:assign --permsetname Animal_Adoption --targetusername <username>

## Update the Schema Using Object Manager or Schema Builder
Use Object Manager or Schema Builder to create new fields and objects or make changes to the schema.

Open the scratch org in your web browser.

In [ ]:
! sfdx force:org:open -u AnimalAdoptionScratch

Once you are happy with the schema and the user access settings, pull the changes to the local project.

In [ ]:
! sfdx force:source:pull

# Demo: Database Operations
After creating a new object, Salesforce automatically creates
* a UI for the object, 
* an Apex class for the object, and 
* an API for the object

There are many tools for managing data in Salesforce via the API. The flow is similar for all of them:
1. Download and install the tool to your local machine
1. Authenticate to the Salesforce instance from the tool
1. Perform database operations

## Download and Install a Data Management Tool
[Dataloader](https://developer.salesforce.com/docs/atlas.en-us.dataLoader.meta/dataLoader/loader_install_general.htm) is developed and maintained by Salesforce and is one of the most popular free options.

In this demo we are using open source tools called [Snowfakery & CumulusCI](https://cumulusci.readthedocs.io/en/stable/get_started.html#install-cumulusci) to generate fake data and load the data into the scratch org.

## Authenticate the Tool to the Scratch Org
CumulusCI comes with Snowfakery and is designed to work with the Salesforce CLI. Run the following command to grant CumulusCI to the scratch org.

In [ ]:
! cci org import AnimalAdoptionScratch sql-guide-to-salesforce__dev

## Perform Database Operations
In this scenario we need to generate some data for each of our objects and insert all of the data into Saleforce.

The instructions for how to generate the data are documented in datasets/salesforce-sample-data.yml

Run the following command to create fake data and insert the data into the scratch org.

In [ ]:
! cci task run generate_and_load_from_yaml -o generator_yaml datasets/salesforce-sample-data.yml --org sql-guide-to-salesforce__dev

Check that the data loaded as expected.

In [ ]:
! sfdx force:data:soql:query -q "SELECT Fields(All) FROM Animal__c LIMIT 10"

In [ ]:
! sfdx force:data:soql:query -q "SELECT FirstName, LastName, Email, Phone, MailingCity, MailingState FROM Contact"

# Demo: Queries
Types of joins
## Inner Join
Returns a row for each value that exists in both tables. Which animals have been matched with an owner?

<img src="assets/inner-join.png" alt="inner join" width="200"/>

In [ ]:
! sfdx force:data:soql:query -q "SELECT Animal_Owner__c, Animal_Type__c, Id, Name, Shelter__r.Name, Animal_Owner__r.FirstName FROM Animal__c WHERE Animal_Owner__c IN (SELECT Id FROM Contact)"

## Left Join
Returns all records from left table with matching values from right table. List all animals with their owner if they have one.

<img src="assets/left-join.png" alt="left join" width="200"/>

In [ ]:
! sfdx force:data:soql:query -q "SELECT Animal_Type__c, Id, Name, Shelter__r.Name, Animal_Owner__r.FirstName, Animal_Owner__r.Email FROM Animal__c"

## Right Join
Returns all records from the right table with matching values from the left table. List all owners with their animals if they have one.

<img src="assets/right-join.png" alt="right join" width="200"/>

In [ ]:
! sfdx force:data:soql:query -q "SELECT FirstName, LastName, Phone, Email, MailingState, (SELECT Animal_Type__c, Id, Name FROM Animals__r) FROM Contact"

## Other Joins
More complicated joins, such as full outer joins or joins on non-relationship fields, can be done in Apex by creating a custom class to model the data structure and using mulitple queries to set values in an instance of the class.